In [1]:
from random import choices
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix 
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import time
import matplotlib.animation as animation

In [2]:
data = pd.read_csv("classification/dataset.csv")
data = data.drop(["Unnamed: 0","presenceP"],axis=1)
data

,idRestaurant,date,Type,Plat,meteo,Event,idEtudiant,Heberg,Rev_par,Bourse,presence
0,Esi016,01-09-2019,midi,Lentilles,Ensoleillé,Normal,17/0001,interne,56879,Oui,1
1,Esi016,01-09-2019,midi,Lentilles,Ensoleillé,Normal,17/0002,externe,19184,Oui,1
2,Esi016,01-09-2019,midi,Lentilles,Ensoleillé,Normal,17/0003,externe,56879,Oui,1
3,Esi016,01-09-2019,midi,Lentilles,Ensoleillé,Normal,17/0004,externe,19184,Oui,1
4,Esi016,01-09-2019,midi,Lentilles,Ensoleillé,Normal,17/0005,interne,139936,Non,0
...,...,...,...,...,...,...,...,...,...,...,...
215995,Esi016,15-08-2022,midi,Loubia,Ensoleillé,Normal,17/0196,interne,25251,Oui,1
215996,Esi016,15-08-2022,midi,Loubia,Ensoleillé,Normal,17/0197,interne,19184,Oui,1
215997,Esi016,15-08-2022,midi,Loubia,Ensoleillé,Normal,17/0198,interne,56879,Oui,0
215998,Esi016,15-08-2022,midi,Loubia,Ensoleillé,Normal,17/0199,interne,56879,Oui,0


In [3]:
ListDate = data['date']

In [4]:
data = data.drop(["idRestaurant", "date", "Type" , "idEtudiant"],axis=1)

In [5]:
data["Plat"].replace(["Lentilles","Loubia","Riz","Spaghetti","Couscous","Weekend"],[0,1,2,3,4,5],inplace=True)
data["meteo"].replace(["Ensoleillé","Nuageux","Mauvais"],[0,1,2],inplace=True)
data["Event"].replace(["Normal","Weekend", "Jour férié","Ramadan"],[0,1,2,3],inplace=True)
data["Heberg"].replace(["interne","externe"],[0,1],inplace=True)
data["Bourse"].replace(["Oui","Non"],[0,1],inplace=True)

In [35]:
data

,Plat,meteo,Event,Heberg,Rev_par,Bourse,presence
0,0,0,0,0,56879,0,1
1,0,0,0,1,19184,0,1
2,0,0,0,1,56879,0,1
3,0,0,0,1,19184,0,1
4,0,0,0,0,139936,1,0
...,...,...,...,...,...,...,...
215995,1,0,0,0,25251,0,1
215996,1,0,0,0,19184,0,1
215997,1,0,0,0,56879,0,0
215998,1,0,0,0,56879,0,0


In [6]:
#data = data.drop(["idRestaurant", "date", "Type" , "idEtudiant", "presenceP",],axis=1)
trainC = data[:144000]
testC = data[144001:]
Ytrain = trainC["presence"]
Xtrain = trainC.drop(["presence"], axis=1)
Ytest = testC["presence"]
Xtest = testC.drop(["presence"], axis=1)

In [7]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [95]:
def initialize_with_zeros(dim):
    w = np.zeros(shape=(dim, 6), dtype='uint8')
    w.nbytes
    b = 0
    return w, b

In [96]:
initialize_with_zeros(144000)[0]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=uint8)

In [100]:
def propagate(w, b, X, Y):
    m = X.shape[1]
    # FORWARD PROPAGATION (FROM X TO COST)
    A = sigmoid(np.dot(w, X.T) + b) # compute activation
    cost = (- 1 / m) * np.sum(Y * np.log(A) + (1 - Y) * (np.log(1 - A))) #compute cost
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = (1 / m) * np.dot(X, (A - Y).T)
    db = (1 / m) * np.sum(A - Y)
    grads = {"dw": dw,
    "db": db}
    return grads, cost

In [99]:
propagate(initialize_with_zeros(720)[0],0,Xtrain,Ytrain)

array([[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       ...,
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]])

In [101]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost =False):
    costs = []
    for i in range(num_iterations) :
        # Cost and gradient calculation
        grads, cost = propagate(w, b, X, Y)
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        # update rule
        w = w - learning_rate * dw # need to broadcast
        b = b - learning_rate * db
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
    params = {"w": w,
    "b": b}
    grads = {"dw": dw,
    "db": db}
    return params, grads, costs

In [102]:
def predict(w, b, X):
    m = X.shape[0]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 6)
    X = X.reshape(6,X.shape[0])
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid(np.dot(w, X.T) + b)
    for i in range(A.shape[1]):
    # Convert probabilities A[0,i] to actual predictions p[0,i]
        Y_prediction[0,i] = 1 if A[0,i] > 0.5 else 0
        return Y_prediction

In [43]:
m = Xtrain.shape[0]
Y_prediction = np.zeros((1,m))
Y_prediction

array([[0., 0., 0., ..., 0., 0., 0.]])

In [103]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_with_zeros(X_train.shape[0])
    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = optimize(w, b, X_train, Y_train,num_iterations, learning_rate, print_cost)
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    # Print train/test Errors
    print("train accuracy: {} %".format(100 -np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 -np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    d = {"costs": costs,
        "Y_prediction_test": Y_prediction_test,
        "Y_prediction_train" : Y_prediction_train,
        "w" : w,
        "b" : b,
        "learning_rate" : learning_rate,
        "num_iterations": num_iterations}
    return d

In [31]:
w, b = initialize_with_zeros(Xtrain.shape[0])

In [37]:
Xtrain.shape[0]

144000

In [104]:
d = model(Xtrain, Ytrain, Xtest, Ytest, num_iterations= 2000, learning_rate = 0.005, print_cost = True)

MemoryError: Unable to allocate 154. GiB for an array with shape (144000, 144000) and data type int64